# Compression de texte

L'idée est de remplacer les caractères du texte par des **codes de longueurs variables** (on parle donc de *codage*) en réservant :
- les codes courts pour les caractères couramment utilisés (ex: `'e'`, `'a'`, `' '`....).
- les codes plus longs pour les caractères peu utilisés (ex: `'z'`, `'w'`...).

Sommaire:
- [1. Fréquences d'occurrence](#freq)
- [2. Codage des caractères](#codage)
    - [a. Arbre de Huffman](#arbre)
    - [b. Table de correspondance](#lut)
    - [c. Compression du texte](#compression)
- [3. Projet Python](#projet)

## 1. Fréquences d'occurrence <a id="freq"></a>

Pour mettre en place ce codage, il est essentiel de connaître les fréquences d'occurrence de chaque caractère. Nous allons travailler, dans un premier temps, sur le texte suivant:

In [ ]:
texte = """
   Un jour, Cunégonde, en se promenant
   auprès du château, dans le petit bois qu'on appelait
   parc, vit entre des broussailles le docteur Pangloss qui donnait
   une leçon de physique expérimentale à la
   femme de chambre de sa mère, petite brune très jolie
   et très docile. Comme Mlle Cunégonde avait beaucoup
   de dispositions pour les sciences, elle observa, sans souffler,
   les expériences réitérées dont elle
   fut témoin; elle vit clairement la raison suffisante du
   docteur, les effets et les causes, et s'en retourna tout
   agitée, toute pensive, toute remplie du désir
   d'être savante, songeant qu'elle pourrait bien être la
   raison suffisante du jeune Candide, qui pouvait aussi être
   la sienne.
   """

**Questionnement:** 
- Quel est le nombre `nb_tot` de caractères constituant ce texte ?

In [ ]:
nb_tot = len(texte)
nb_tot

- Quel type de codage est actuellement utilisé pour ces caractères ?

In [ ]:
import sys
sys.getdefaultencoding()

In [ ]:
len(texte.encode('utf-8'))

*Pour améliorer les performances de compression, nous n'allons créer des codes que pour les caractères qui apparaissent dans le texte.*

- Est-il préférable de stocker la liste des caractères dans un *tableau* (type `list` en Python) ou une structure chaînée ?

Une structure chaînée car le nombre d'éléments varie

*(recopier ou importer la classe `Liste` du document ressource et identifier les méthodes utiles)*

In [ ]:
from tp_compression import Liste
help(Liste)

- De combien de caractères différents `nb_carac_diff` est constitué le texte ? Stocker ces caractères dans une *Liste* `liste_carac`

In [ ]:
liste_carac = Liste()
for c in texte:
    idx = liste_carac.index(c)
    
    if idx==-1: # nouveau caractère
        liste_carac.append(c)
        
liste_carac.length(), liste_carac

- Créer et compléter une nouvelle *Liste* `liste_occurrences` où chaque élément est le nombre d'occurrences du caractère correspondant dans `liste_carac`. *(vérifier que la somme de tous ces nombres vaut `nb_tot`)*

In [ ]:
liste_carac = Liste()
liste_occurrences = Liste()
for c in texte:
    idx = liste_carac.index(c)
    
    if idx==-1: # nouveau caractère
        liste_carac.append(c)
        liste_occurrences.append(1)
    else: # déjà rencontré
        val = liste_occurrences.element(idx)
        liste_occurrences.replace(idx, val+1)
liste_occurrences.length(), liste_occurrences

In [ ]:
# Calcul de la somme
somme = 0
# À compléter
for i in range(liste_occurrences.length()):
    somme += liste_occurrences.element(i)
print(somme)

assert somme==nb_tot, "La liste d'occurrences n'est pas correctement remplie"

On définit la fréquence d'occurrence d'un caractère *c* par 
$$f(c)=\frac{\text{nb_occurrance(c)}}{\text{nb_tot}}$$

- Créer un tableau `freq` (type `list` de Python) où chaque élément est un p-uplet au format `(c, f(c))`

In [ ]:
freq = [None] * liste_occurrences.length()

for i in range(liste_occurrences.length()):
    c = liste_carac.element(i)
    f = liste_occurrences.element(i)/nb_tot
    freq[i] = (c, f)
    
freq[:5]

- (Optionnel) Trier cette liste (avec la méthode de votre choix) selon les fréquences d'occurrence décroissantes. Quels sont les 5 caractères qui apparaissent le plus souvent ?

In [ ]:
help(sorted)

In [ ]:
def ma_cle(x):
    return x[1]

freq2 = sorted(freq, key=ma_cle, reverse=True)
freq2[:5]

## 2. Codage des caractères <a id="codage"></a>

Le code de chaque caractère correspond à son chemin absolu dans l'arbre ci-après. Par convention, on lit un chemin en ajoutant :
- `0` si on se déplace vers l'enfant gauche.
- `1` si on se déplace vers l'enfant droit.

![Arbre de Huffman ](https://snlpdo.fr/tnsi/img/04-ex_arbre_huffman.png)

Exemples:
- Le caractère `'e'` est remplacé par le code `100` (=3 bits).
- Le caractère `'a'` est remplacé par le code `11111` (=5 bits)

**Questionnement:**
- Comment appelle-t-on les n&oelig;uds qui contiennent les caractères dans cet arbre ?

des feuilles

- Quel code remplace la portion de texte `Un jour` en utilisant cet arbre ?

In [ ]:
code = lut['U']+lut['n']+lut[' ']+lut['j']+lut['o']+lut['u']+lut['r']

- Combien de bits fait ce code ? Combien d'octets faut-il pour le stocker ?

In [ ]:
from math import ceil

len(code), ceil(len(code)/8)

### a. Construction de l'arbre <a id="arbre"></a>

Pour compresser le plus possible, il faut que les caractères les plus (resp. moins) utilisés soient situés sur les feuilles les plus proches (resp. éloignées) de la racine.

L'arbre de Huffman peut s'obtenir en appliquant l'algorithme suivant:

<div class="alert alert-block alert-danger">
    
1. Pour chaque caractère `c`, on crée un nouvel arbre (d'un seul n&oelig;ud) dont la racine vaut `(c, f(c))`.

2. On fusionne les 2 arbres de fréquences minimales `f(c1)` et `f(c2)` en un seul nouvel arbre dont:
    - les enfants sont les 2 arbres précédents,
    - la racine a pour nouvelle fréquence `f(c1)+f(c2)` (le nouveau caractère, pourra s'appeler `c1+c2`).

3. On répète l'opération 2 jusqu'à n'avoir plus qu'un seul arbre.

*(Recopier ou importer au préalable la classe `Arbre` fournie dans le document ressources et identifier les méthodes utiles pour cette partie)*

In [ ]:
from tp_compression import Arbre
help(Arbre)

**Questionnement :**
- **Étape 1** : remplir une Liste `liste_arbre` avec tous les arbres à 1 n&oelig;ud (1 par caractère).

In [ ]:
liste_arbres = Liste()
for element in freq:
    arbre = Arbre( element, None, None)
    liste_arbres.append(arbre)
    
liste_arbres.length(), liste_arbres

- **Étape 2a** : Écrire la fonction `extract_min` qui extrait l'élément minimal d'une telle `Liste` (attention: les éléments de cette liste sont des Arbres et le tri se fait sur la 2ème valeur du tuple de leur racine)

In [ ]:
def extract_min(liste):
    # initialisation
    i_min = 0
    f_min = liste.element(0).value()[1]
    
    # le reste de la liste
    for i in range(1, liste.length()):
        f = liste.element(i).value()[1]
        if f<f_min:
            i_min = i
            f_min = f
            
    arbre = liste.element(i_min)
    liste.remove(i_min)
    return arbre

In [ ]:
# Vérification
#a = extract_min(liste_arbres)
#assert a.root[0] == 'U', "Il ne s'agit pas du bon minimum"
#assert liste_arbres.length()==38, "Il faut supprimer le minimum de la liste des arbres"

- **Étapes 2b et 3** : fusionner tous les arbres de cette liste selon l'algorithme de Huffman

In [ ]:
# Algorithme de Huffman
while liste_arbres.length()>1: # tant qu'il reste plus d'un noeud
    # Extraire le 1er arbre de fréquence minimale
    arbre1 = extract_min(liste_arbres)
    # Extraire le 2ème arbre de fréquence minimale
    arbre2 = extract_min(liste_arbres)
    # Fusionner ces 2 arbres pour créer en un nouveau
    c = arbre1.value()[0]+arbre2.value()[0]
    f = arbre1.value()[1]+arbre2.value()[1]
    arbre = Arbre( (c,f) , arbre1, arbre2)
    # Ajouter ce nouvel arbre dans la liste
    liste_arbres.append(arbre)

In [ ]:
# Dernier arbre restant
arbre_huffman = liste_arbres.element(0)

Que valent la hauteur et la taille de `arbre_huffman` ?

In [ ]:
arbre_huffman.height(), arbre_huffman.size()

### b. Table de correspondance <a id="lut"></a>

Pour accélérer la phase de codage, on va remplir un *dictionnaire* Python `lut` où les **clés** sont les caractères du texte et les **valeurs** sont les codes de Huffman correspondants.

Exemple: `lut['e'] = '100'`

**Questionnement :**

- Quel type de parcours d'arbre permet d'identifier le chemin de chaque caractère ?

parcours en profondeur d'abord

- Mettre en &oelig;uvre ce parcours pour remplir la table de correspondance `lut`:

In [ ]:
lut = {}
def parcours_postfixe(a, code):
    """
    arbre gauche, puis arbre droit, puis valeur
    """
    if a.is_empty(): 
        return
    
    if a.has_left():
        parcours_postfixe(a.left(), code+'0')
    if a.has_right():
        parcours_postfixe(a.right(), code+'1')
    
    if not(a.has_left()) and not(a.has_right()): # une feuille
        lut[a.value()[0]] = code
    
parcours_postfixe(arbre_huffman, '')

### c. Compression du texte <a id="compression"></a>

- Utiliser la table de correspondance précédente pour générer le code correspondant au texte fourni au début de ce document

In [ ]:
code = ''
for c in texte:
    lut[c]
    


- Quel taux de compression obtient-t-on ? (on suppose que le code obtenu est sauvegardé en binaire)

In [ ]:
750 / ceil(len(code)/8)

## 3. Projet Python 

(pour 1 binôme)

**Élève 1 :**

Mettre au point un codeur qui:
- créé l'arbre de Huffman d'un texte quelconque.
- génère un fichier avec:
    - le code obtenu pour ce texte (au format binaire)
    - Le contenu de l'arbre (indispensable pour le décodeur)
- affiche le taux de compression final.
        
**Éleve 2 :**

Mettre au point un décodeur qui:
- récupère l'arbre utilisé par le codeur.
- décompresse le texte.
- vérifie qu'il n'y a pas eu de perte.